In [ ]:
from utils.binaries import *

In [ ]:
runlist =  pd.read_csv(f'/cr/data01/filip/xy-calibration/config/calib_runlists/calib_runs_2022-10.list', sep=';')
runlist['comment'] = [comment.strip() for comment in runlist['comment']]
runlist['source'] = [source.strip() for source in runlist['source']]


for month in ['2023-10', '2023-11']:
    this_runlist = pd.read_csv(f'/cr/data01/filip/xy-calibration/config/calib_runlists/calib_runs_{month}.list', sep=';')
    this_runlist['comment'] = [comment.strip() for comment in this_runlist['comment']]
    this_runlist['source'] = [source.strip() for source in this_runlist['source']]

    runlist = pd.concat([runlist, this_runlist])



In [ ]:
runlist

In [ ]:
runlist['source'].keys()

In [ ]:
only_xy_runs = runlist
# only_xy_runs = runlist.drop(runlist[runlist['comment'] != '""'].index)
only_xy_runs = runlist.drop(runlist[runlist['source'] != '"OLO"'].index)
# only_xy_runs = runlist.drop(runlist[runlist['forDB'] == 0].index)
only_xy_runs

In [ ]:
XY_measurements = {}

for index, measurement in only_xy_runs.iterrows():

    try:
        _ = XY_measurements[measurement['telescope'].strip()]
    except KeyError:
        XY_measurements[measurement['telescope'].strip()] = []

    (runid, telescope), date = measurement[:2], measurement['date'].strip()

    same_day = runlist[[d.strip() == date for d in runlist['date']]]
    same_telescope = same_day[[t.strip() == telescope.strip() for t in same_day['telescope']]]

    year, month, day = date.split('-')

    XY_measurement = {'XY' : measurement['#runid'],
                      'CalA_open_shutter' : [None, None], 
                      'date' : f"{year}{month}".replace('10','oct').replace('11','nov')}

    # find Cal A closed shutter
    for candidate_index, candidate in same_telescope.iterrows():
        if candidate['comment'] == '"Cal A"':
            if candidate_index < index: XY_measurement['CalA_closed_shutter'][0] = candidate['#runid']
            if candidate_index > index: XY_measurement['CalA_closed_shutter'][1] = candidate['#runid']
        if candidate['comment'] == '"Cal A open shutter"':
            if candidate_index < index: XY_measurement['CalA_open_shutter'][0] = candidate['#runid']
            if candidate_index > index: XY_measurement['CalA_open_shutter'][1] = candidate['#runid']

    XY_measurements[telescope.strip()].append(XY_measurement)

In [ ]:
XY_measurements

In [ ]:
import pickle

# save events to disk
with open('/cr/users/filip/Projects/xy-reproducibility/reproducibility_dataset.pkl', "wb") as f:
    pickle.dump(XY_measurements, f, pickle.HIGHEST_PROTOCOL)